This is a demo file illustrating how to use the functions in **EigenvalueSolver.jl**.

In [5]:
include("../src/EigenvalueSolver.jl")
using DynamicPolynomials

Construct a polynomial system ${\cal F} = (f_1, f_2)$ given by two plane curves of degree 20. 

In [8]:
@polyvar x[1:2]
ds = [20;20]
f = EigenvalueSolver.getRandomSystem_dense(x,ds)

2-element Array{Polynomial{true,Float64},1}:
 0.6409659107655169x₁²⁰ - 0.4696338882060303x₁¹⁹x₂ + 0.9852768608144051x₁¹⁸x₂² - 0.15852168622409454x₁¹⁷x₂³ - 1.2466271484720017x₁¹⁶x₂⁴ - 0.5528020750802468x₁¹⁵x₂⁵ + 0.7856569448293725x₁¹⁴x₂⁶ - 0.6831170689934523x₁¹³x₂⁷ - 0.17083106789456287x₁¹²x₂⁸ + 0.6672310855349919x₁¹¹x₂⁹ - 1.6673235481094049x₁¹⁰x₂¹⁰ - 0.6305220295306038x₁⁹x₂¹¹ + 0.08226023941819009x₁⁸x₂¹² - 0.27321263245123056x₁⁷x₂¹³ + 0.7690223778108577x₁⁶x₂¹⁴ + 0.8925437868591567x₁⁵x₂¹⁵ - 0.5265691253691919x₁⁴x₂¹⁶ + 0.18766464902361496x₁³x₂¹⁷ - 0.48471270170875275x₁²x₂¹⁸ + 0.8841418299702113x₁x₂¹⁹ + 0.41058280194708296x₂²⁰ - 0.9957122759783467x₁¹⁹ + 0.2020406761293582x₁¹⁸x₂ + 0.5529234426553635x₁¹⁷x₂² + 1.2264532895921096x₁¹⁶x₂³ - 0.27416065830821285x₁¹⁵x₂⁴ - 0.7702411446133729x₁¹⁴x₂⁵ + 0.23220675699022214x₁¹³x₂⁶ - 0.10788286349682548x₁¹²x₂⁷ + 1.7860394315146375x₁¹¹x₂⁸ + 1.0273722868245665x₁¹⁰x₂⁹ - 1.0866531589397928x₁⁹x₂¹⁰ - 2.621178508094653x₁⁸x₂¹¹ + 0.8929010405063591x₁⁷x₂¹² + 0.30

Compute all intersection points of these two curves.

In [9]:
@time sol = EigenvalueSolver.solve_CI_dense(f,x; DBD = false)

constructing resultant map...
  0.092083 seconds (494.63 k allocations: 45.765 MiB)
       Sylv is a matrix of size (820, 420)
computing cokernel...
  0.169533 seconds (13 allocations: 14.539 MiB, 19.84% gc time)
       relative size of last nz singular value = 0.05016347272879749
N has size (400, 820)
finding a basis using QR-P...
  0.125630 seconds (27.32 k allocations: 44.948 MiB)
extracting relevant eigenvalues...
  0.634813 seconds (10.47 k allocations: 33.618 MiB, 2.99% gc time)
inverting monomial map given by A₀...
  0.000562 seconds (1.25 k allocations: 237.844 KiB)
found 400 solutions
  1.118354 seconds (682.36 k allocations: 156.470 MiB, 4.71% gc time)


400-element Array{Array{Complex{Float64},1},1}:
 [0.3283513106375871 + 0.0im, 0.21968242557330245 + 0.0im]
 [0.8340645573737837 - 0.15147766338917534im, -0.9084321641449856 + 0.3759690322118544im]
 [0.834064557373795 + 0.1514776633891762im, -0.9084321641450148 - 0.37596903221185796im]
 [0.838435570221846 + 0.0im, -1.4101012771483301 + 1.7268760155332973e-16im]
 [0.3024815591651316 - 0.30486636046574445im, 0.6276687498648658 + 0.6899453714711581im]
 [0.302481559165133 + 0.30486636046574495im, 0.6276687498648624 - 0.6899453714711593im]
 [0.7260163157226573 - 0.10253427099782869im, -0.5252420667791976 + 0.6907196030977429im]
 [0.7260163157226609 + 0.10253427099782023im, -0.5252420667792133 - 0.6907196030977182im]
 [0.48157413473022154 + 0.2289684630890175im, -0.5719677427209288 - 0.6188269517228203im]
 [0.4815741347302077 - 0.2289684630890147im, -0.5719677427208906 + 0.6188269517228128im]
 [0.8283584650000284 - 0.0919038442567403im, -0.8689837660803517 - 0.3590753496253633im]
 [0.82835846

Compute the backward error of all computed solutions

In [13]:
BWEs = EigenvalueSolver.get_residual(f,sol,x)
BWE = maximum(BWEs)

1.8584211442640938e-11

We now solve a dense, 3-dimensional system given by equations of degree 4, 8 and 12. 

In [16]:
@polyvar x[1:3]
ds = [4;8;12] # degrees of the equations
f = EigenvalueSolver.getRandomSystem_dense(x,ds)
@time sol = EigenvalueSolver.solve_CI_dense(f,x)
BWEs = EigenvalueSolver.get_residual(f,sol,x) # compute the relative backward error of all solutions
maximum(BWEs)

Computing cokernel degree by degree...
degree = 13
degree = 14
degree = 15
degree = 16
degree = 17
degree = 18
degree = 19
degree = 20
degree = 21
degree = 22
N has size (384, 2300)
finding a basis using QR-P...
  0.452805 seconds (85.47 k allocations: 156.293 MiB, 6.58% gc time)
extracting relevant eigenvalues...
  0.879298 seconds (13.14 k allocations: 37.899 MiB, 2.78% gc time)
inverting monomial map given by A₀...
  0.000655 seconds (1.21 k allocations: 282.844 KiB)
found 384 solutions
  4.954755 seconds (3.32 M allocations: 975.533 MiB, 4.21% gc time)


2.368989367423086e-13

The package provides a specialized function for unmixed systems

In [18]:
@polyvar x[1:2];
A = [0 0; 1 0; 1 1; 0 1; 2 2] # monomials whose convex hull is the polytope P
d = [5;12] # equation i has support inside d[i]*P
f = EigenvalueSolver.getRandomSystem_unmixed(x,A,d)
@time sol, A₀, E, D = EigenvalueSolver.solve_CI_unmixed(f,x,A,d)

constructing resultant map...
  0.043151 seconds (254.06 k allocations: 26.837 MiB)
       Sylv is a matrix of size (685, 450)
computing cokernel...
  0.252654 seconds (8.73 k allocations: 23.483 MiB, 6.82% gc time)
       rank = 445
       relative size of last nz singular value = 0.03951148417622439
       gap = 1.5778992995074556e14
N has size (240, 685)
Checking criterion...
********* criterion satisfied ********* γ = 240
finding a basis using QR-P...
  0.134559 seconds (33.82 k allocations: 58.849 MiB, 7.27% gc time)
extracting relevant eigenvalues...
  0.347086 seconds (20.23 k allocations: 38.866 MiB, 5.01% gc time)
inverting monomial map given by A₀...
  0.000488 seconds (1.03 k allocations: 175.734 KiB)
found 240 solutions
  1.280224 seconds (457.19 k allocations: 169.884 MiB, 3.47% gc time)


(Array{Complex{Float64},1}[[0.12266421556104042 - 0.2909976804328231im, 7.421154131155326 + 4.475236325910658im], [0.7924221375297856 - 0.5792969637108061im, 0.21367402596591054 - 1.0305578280165102im], [0.7617898359926111 - 0.5684156983109347im, 0.9968841077823648 + 0.44891115011692606im], [-0.2210851444946813 + 0.7236508018037806im, 1.0449363762492112 + 0.364448888007402im], [-0.5239505652120644 + 0.41771558572457973im, 1.7331301242280828 + 0.28068635549910737im], [-0.34658823864732424 - 0.9960125458783048im, 0.4647212176339789 - 0.8311427782963454im], [-0.9429766595529236 + 0.20121063855769644im, 1.0246734842361862 - 0.20443517410678524im], [-1.3300877199768772 + 0.3095813319329557im, 0.7690502990912333 - 0.13116436830744962im], [-0.37852221026107175 - 1.0380564942267703im, 0.5973677208349842 - 0.6364949823636955im], [-1.1948786721127205 - 0.5467627034663256im, 0.6609992110094236 - 0.36453943994607896im]  …  [0.1877675211783758 + 0.9490126210965348im, -0.2695011542274712 + 0.9703077

The function returns an admissible tuple, which can be re-used by plugging it into the function **solve_EV**.

In [19]:
sol = EigenvalueSolver.solve_EV(f, x, A₀, E, D)

constructing resultant map...
  0.048234 seconds (254.06 k allocations: 26.837 MiB)
       Sylv is a matrix of size (685, 450)
computing cokernel...
  0.344198 seconds (15 allocations: 23.350 MiB, 7.21% gc time)
       rank = 445
       relative size of last nz singular value = 0.03951148417622439
       gap = 1.5778992995074556e14
N has size (240, 685)
Checking criterion...
********* criterion satisfied ********* γ = 240
finding a basis using QR-P...
  0.175915 seconds (33.82 k allocations: 58.849 MiB, 11.51% gc time)
extracting relevant eigenvalues...
  0.373944 seconds (20.26 k allocations: 39.050 MiB, 6.45% gc time)
inverting monomial map given by A₀...
  0.000677 seconds (1.03 k allocations: 175.734 KiB)
found 240 solutions


240-element Array{Array{Complex{Float64},1},1}:
 [0.9400701523483251 - 0.19179858919466697im, -1.5421537938522003 + 0.6632098177619461im]
 [0.49480032521598166 + 0.42205269550555613im, -0.3083600153992075 - 0.956844903286721im]
 [-1.3213420182014808 - 0.10416880414704961im, 0.3194950443654679 + 0.8290573266518774im]
 [1.2077624615673694 + 0.38806169874432095im, -0.5782514457774244 - 0.9141798864160151im]
 [0.6438873186495029 - 0.5162432555591691im, -1.1524856789501419 - 0.021730887800279586im]
 [0.32510955647189965 - 1.0155250508921587im, -0.6112960716149201 + 0.4266234847250526im]
 [-1.1948786721126459 + 0.5467627034663787im, 0.6609992110094451 + 0.36453943994608984im]
 [0.7900309098770867 - 0.5277530118047016im, -1.0434103554702328 - 0.40297013888331834im]
 [0.6038632766952675 - 0.7913401093050683im, -1.084803511627261 + 0.38896788239261126im]
 [1.6848865532906931 + 0.19256740407183331im, -0.44287234342546666 - 0.8127924129504892im]
 [0.4969528957354097 - 0.7295662458186498im, -1.093

Here's how to solve a square, multi-graded dense system.

In [23]:
@polyvar x[1:4]
vargroups = [[x[1:2]];[x[3:4]]] # groups of variables
ds = [1 6;2 1;3 2;4 1] # multidegrees of the 4 equations in the 2 variable groups
f = EigenvalueSolver.getRandomSystem_multi_dense(vargroups,ds)
@time sol = EigenvalueSolver.solve_CI_multi_dense(f,vargroups,ds)
residuals = EigenvalueSolver.get_residual(f,sol,x)
maximum(residuals)

constructing resultant map...
  0.268783 seconds (908.37 k allocations: 180.977 MiB, 12.14% gc time)
       Sylv is a matrix of size (3025, 4023)
computing cokernel...
compressing Sylv...
  1.275672 seconds (4 allocations: 162.660 MiB, 4.32% gc time)
 23.166801 seconds (14 allocations: 419.252 MiB, 1.82% gc time)
       rank = 2806
       relative size of last nz singular value = 0.0019188818470385772
       gap = 4.032549427829967e12
N has size (219, 3025)
finding a basis using QR-P...
  0.225950 seconds (190.58 k allocations: 126.337 MiB)
extracting relevant eigenvalues...
  0.279287 seconds (16.34 k allocations: 23.902 MiB)
inverting monomial map given by A₀...
  0.000370 seconds (739 allocations: 246.938 KiB)
found 219 solutions
 26.109526 seconds (3.12 M allocations: 1.007 GiB, 2.13% gc time)


1.0123046110531718e-10

Here's how to solve a square, multi-unmixed system.

In [24]:
@polyvar x[1:4]
A1 = [0 0; 1 0; 1 1; 0 1; 2 2]
A2 = 2*[0 0; 1 0; 0 1]
sups = [[A1]; [A2]]
vargroups = [[x[1:2]];[x[3:4]]]
ds = ones(Int,4,2)
f = EigenvalueSolver.getRandomSystem_multi_unmixed(vargroups,sups,ds;complex = false)
@time sol, A₀, E, D = EigenvalueSolver.solve_CI_multi_unmixed(f,vargroups,sups,ds)
residuals = EigenvalueSolver.get_residual(f,sol,x)
maximum(residuals)

constructing resultant map...
  0.218957 seconds (750.66 k allocations: 169.086 MiB, 8.72% gc time)
       Sylv is a matrix of size (2745, 4592)
computing cokernel...
compressing Sylv...
  1.229702 seconds (4 allocations: 153.657 MiB, 1.79% gc time)
 17.706569 seconds (14 allocations: 345.262 MiB, 0.35% gc time)
       rank = 2649
       relative size of last nz singular value = 0.0019703991868776807
       gap = 4.2611966538667666e12
N has size (96, 2745)
finding a basis using QR-P...
  0.191508 seconds (335.68 k allocations: 112.554 MiB)
extracting relevant eigenvalues...
  0.084875 seconds (23.57 k allocations: 14.423 MiB)
inverting monomial map given by A₀...
  0.000688 seconds (379 allocations: 252.016 KiB)
found 96 solutions
 21.656844 seconds (5.27 M allocations: 981.868 MiB, 1.26% gc time)


1.8528576347932377e-12

The following is an example of a mixed system, coming from molecular biology.

In [27]:
@polyvar t[1:3]
β = [-13 -1 -1 24 -1; -13 -1 -1 24 -1; -13 -1 -1 24 -1]
mons1 = [1 t[2]^2 t[3]^2 t[2]*t[3] t[2]^2*t[3]^2]
mons2 = [1 t[3]^2 t[1]^2 t[3]*t[1] t[3]^2*t[1]^2]
mons3 = [1 t[1]^2 t[2]^2 t[1]*t[2] t[1]^2*t[2]^2]
f = [β[1,:]'*mons1';β[2,:]'*mons2';β[3,:]'*mons3'][:]
@time sol, A₀, E, D = EigenvalueSolver.solve_CI_mixed(f,t)
residuals = EigenvalueSolver.get_residual(f,sol,t)
maximum(residuals)
@time EigenvalueSolver.solve_EV(f,x,A₀,E,D;check_criterion = false)

computed the sum of 2 out of 4 polytopes
computed the sum of 3 out of 4 polytopes
computed the sum of 4 out of 4 polytopes
computed the sum of 2 out of 3 polytopes
computed the sum of 3 out of 3 polytopes
computed the sum of 2 out of 3 polytopes
computed the sum of 3 out of 3 polytopes
computed the sum of 2 out of 3 polytopes
computed the sum of 3 out of 3 polytopes
computed the sum of 2 out of 3 polytopes
computed the sum of 3 out of 3 polytopes
constructing resultant map...
  0.002848 seconds (13.82 k allocations: 1.555 MiB)
       Sylv is a matrix of size (200, 252)
computing cokernel...
compressing Sylv...
  0.001393 seconds (4 allocations: 706.406 KiB)
  0.016524 seconds (12 allocations: 1.856 MiB)
       rank = 184
       relative size of last nz singular value = 0.0007685204395029549
       gap = 2.303798564387252e12
N has size (16, 200)
Checking criterion...
********* criterion satisfied ********* γ = 16
finding a basis using QR-P...
  0.001674 seconds (6.29 k allocations: 965.

16-element Array{Array{Complex{Float64},1},1}:
 [10.857703599626547 - 1.593141052371809e-15im, 0.7795480450791494 - 3.219472543334698e-15im, 0.7795480450791681 + 7.633800875948252e-16im]
 [-10.857703599626417 - 5.919115334406862e-14im, -0.7795480450791803 - 1.480222603336276e-15im, -0.7795480450791594 - 1.7058721995407657e-14im]
 [4.625181601344248 - 4.085183109265233e-15im, 4.625181601344264 - 7.111244671683925e-15im, 0.3320730983656658 - 1.1347730859070094e-16im]
 [-4.625181601344271 + 8.478014433727115e-14im, -0.3320730983656651 + 6.08694050019213e-15im, -4.6251816013442735 + 8.47801443372712e-14im]
 [-4.625181601344421 + 1.7310185377569983e-13im, -4.625181601344446 + 1.7104786053294665e-13im, -0.332073098365678 + 1.2575624014282113e-14im]
 [-4.625181601344205 - 2.983954415836855e-13im, -4.625181601344161 - 3.0044943482643657e-13im, -4.625181601344174 - 3.0250342806919144e-13im]
 [4.625181601344272 - 1.325339519876499e-13im, 4.625181601344245 - 1.3207031497419987e-13im, 4.6251816013

An overdetermined unmixed system in 15 variables with 100 solutions.

In [29]:
using LinearAlgebra
@polyvar x[1:15]
A = Matrix(I,15,15)
A = [A[1:14,:]; sum(A[[14;15],:], dims = 1);zeros(1,15)]
A = Int.(A)
A[14,:] = A[14,:]+A[13,:] 
n = 15
F = EigenvalueSolver.getRandomSystem_unmixed(x, A, [2]; complex = true)[1]
mons = monomials(F)
δ = 100
pts = [randn(ComplexF64,n) for i = 1:δ]
println("the system has $δ solutions")
f, κ = EigenvalueSolver.getVanishingPolynomials(pts, mons, x;augm_prec = false)
α = ones(Int,length(f))*2
R_offline = @timed EigenvalueSolver.solve_OD_unmixed(f, x, A, α;complex = true)

the system has 100 solutions
Looking for an admissible tuple
!!!!!!!!! criterion violated !!!!!!!!!
degree = 3
!!!!!!!!! criterion violated !!!!!!!!!
degree = 4
********* criterion satisfied ********* γ = 100
N has size (100, 3876)
finding a basis using QR-P...
  0.130881 seconds (147.71 k allocations: 88.422 MiB)
extracting relevant eigenvalues...
  0.077620 seconds (12.88 k allocations: 16.160 MiB)
inverting monomial map given by A₀...
  0.000385 seconds (474 allocations: 287.641 KiB)
found 100 solutions


(value = Array{Complex{Float64},1}[[0.36998426298524006 - 0.9035301589480986im, -0.6688551010108201 - 0.45996110314749833im, -0.7004374580072635 - 0.24954443305551566im, -0.7532794047457347 - 0.748339173864237im, -0.9594096171150235 + 1.768947312470154im, 0.806361841899264 - 1.0127159548622913im, 0.438429043580892 - 0.890420859579995im, 0.4503940806853003 + 0.5112943934904911im, -1.015680535255077 - 0.2895866116503972im, -0.574894157966052 + 0.9261098548370041im, 1.1468922328432272 + 0.38552432462929903im, 0.6739898820799893 + 0.18926570731052347im, -0.5360094314479995 + 0.6633433515731271im, 0.5963760049333667 + 0.053152756586274115im, 0.7169665834690812 - 1.0622102607245678im], [0.07280762666816469 + 0.66374171093064im, 0.5805816290801457 - 0.015233831576968913im, -0.0775319517869701 - 0.8520268009458108im, 0.20792914881042052 + 1.3505792915380515im, 0.11493781069826528 - 0.32111945378603074im, 0.7783694490218536 - 0.3184406776310606im, 0.2908577109173867 - 1.680790944150907im, -0.09

In [31]:
EigenvalueSolver.get_residual(f,R_offline.value,x)

100-element Array{Float64,1}:
 2.190790200500412e-15
 3.681628812829275e-15
 1.0851512733028113e-14
 5.674323705664605e-15
 2.220729682799523e-14
 1.2596885579383696e-15
 8.748979998845645e-15
 2.3624593075391487e-15
 5.252777524591009e-15
 1.1391105386685125e-14
 6.583921008910702e-15
 6.431057665429322e-15
 1.855085228900243e-14
 ⋮
 2.580903089624297e-15
 6.880035657416989e-15
 2.3085721537175993e-15
 4.766942853740794e-15
 2.7791494173973944e-15
 3.3391411163496415e-15
 6.950526866237324e-15
 2.0477046898537117e-15
 3.204153896543534e-15
 1.4644439519541162e-15
 3.4737635337873808e-15
 1.5728867552922047e-15